In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering, KMeans
import scipy.cluster.hierarchy as sch

data = pd.read_csv('customer_shopping_data.csv')

In [2]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [3]:
data.head()

invoice_no customer_id  gender  age  category  quantity    price  \
0    I138884     C241288  Female   28  Clothing         5  1500.40   
1    I317333     C111565    Male   21     Shoes         3  1800.51   
2    I127801     C266599    Male   20  Clothing         1   300.08   
3    I173702     C988172  Female   66     Shoes         5  3000.85   
4    I337046     C189076  Female   53     Books         4    60.60   

  payment_method invoice_date   shopping_mall  
0    Credit Card     5/8/2022          Kanyon  
1     Debit Card   12/12/2021  Forum Istanbul  
2           Cash    9/11/2021       Metrocity  
3    Credit Card   16/05/2021    Metropol AVM  
4           Cash   24/10/2021          Kanyon

In [30]:
data.duplicated().sum()

np.int64(0)

In [29]:
data.isnull().sum()

invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
shopping_mall     0
dtype: int64

In [6]:
data = data.dropna()

In [7]:
data.isnull().sum()

invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
shopping_mall     0
dtype: int64

In [8]:
data['invoice_date'] = pd.to_datetime(data['invoice_date'], dayfirst=True)


In [9]:
data.dtypes

invoice_no                object
customer_id               object
gender                    object
age                        int64
category                  object
quantity                   int64
price                    float64
payment_method            object
invoice_date      datetime64[ns]
shopping_mall             object
dtype: object

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns_to_normalize = ['age', 'quantity', 'price']
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])

In [11]:
data[columns_to_normalize].head()

age  quantity     price
0  0.196078      1.00  0.285078
1  0.058824      0.50  0.342299
2  0.039216      0.00  0.056218
3  0.941176      1.00  0.571163
4  0.686275      0.75  0.010557

In [12]:
# --- Prepare for Clustering ---
df_clustering = data.select_dtypes(include=[np.number])

In [13]:
# Split Data (before scaling)
X_train, X_test = train_test_split(df_clustering, test_size=0.3, random_state=42)
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)


Train data shape: (69619, 3)
Test data shape: (29838, 3)


In [14]:
# Scale Data (only fit on training)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [15]:
df_clustering = data.select_dtypes(include=[np.number])
df_clustering.head()

age  quantity     price
0  0.196078      1.00  0.285078
1  0.058824      0.50  0.342299
2  0.039216      0.00  0.056218
3  0.941176      1.00  0.571163
4  0.686275      0.75  0.010557

In [16]:
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans_labels_train = kmeans.fit_predict(X_train_scaled)
silhouette_kmeans_train = silhouette_score(X_train_scaled, kmeans_labels_train)
print(f'Silhouette Score for K-Means (Train): {silhouette_kmeans_train:.2f}')

Silhouette Score for K-Means (Train): 0.31


In [17]:
kmeans_labels_test = kmeans.predict(X_test_scaled)
silhouette_kmeans_test = silhouette_score(X_test_scaled, kmeans_labels_test)
print(f'Silhouette Score for K-Means (Test): {silhouette_kmeans_test:.2f}')

Silhouette Score for K-Means (Test): 0.31


In [18]:

# تحويل مصفوفة NumPy إلى DataFrame
X_train_df = pd.DataFrame(X_train)

# أخذ عينة عشوائية من البيانات
X_train_small = X_train_df.sample(frac=0.1, random_state=42)


In [31]:
X_train_small_scaled = scaler.transform(X_train_small)


In [33]:
plt.figure(figsize=(10, 6))
dendrogram = sch.dendrogram(sch.linkage(X_train_small_scaled, method='ward'))
plt.title("Dendrogram - Hierarchical Clustering")
plt.xlabel("Customers")
plt.ylabel("Euclidean distances")
plt.show()

In [20]:
hierarchical = AgglomerativeClustering(n_clusters=3, linkage='ward')
hierarchical_labels_train = hierarchical.fit_predict(X_train_small)
silhouette_hier_train = silhouette_score(X_train_small, hierarchical_labels_train)
print(f'Silhouette Score for Hierarchical Clustering (Train): {silhouette_hier_train:.2f}')

Silhouette Score for Hierarchical Clustering (Train): 0.32


In [21]:
hierarchical_test = AgglomerativeClustering(n_clusters=3, linkage='ward')
hierarchical_labels_test = hierarchical_test.fit_predict(X_test)
silhouette_hier_test = silhouette_score(X_test, hierarchical_labels_test)
print(f'Silhouette Score for Hierarchical Clustering (Test): {silhouette_hier_test:.2f}')

Silhouette Score for Hierarchical Clustering (Test): 0.32


In [22]:
# for test
cluster_names = {
    0: "Low Spenders",
    1: "High Value Customers",
    2: "Average Shoppers"
}
new_customer = np.array([[10, 1, 1]])
new_customer_scaled = scaler.transform(new_customer)
cluster = kmeans.predict(new_customer_scaled)
print(f"The new customer belongs to cluster: {cluster_names[cluster[0]]}")

The new customer belongs to cluster: Average Shoppers


c:\Users\nadia\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [34]:
print(f'Silhouette Score for KMeans: {silhouette_kmeans_train:.2f}')
print(f'Silhouette Score for Hierarchical Clustering: {silhouette_hier_train:.2f}')


Silhouette Score for KMeans: 0.31
Silhouette Score for Hierarchical Clustering: 0.32


In [37]:
import numpy as np

# توزيع العينات على الكلاسترز
unique_kmeans, counts_kmeans = np.unique(kmeans_labels_train, return_counts=True)
unique_hier, counts_hier = np.unique(hierarchical_labels_train, return_counts=True)

print("KMeans cluster distribution:")
for label, count in zip(unique_kmeans, counts_kmeans):
    print(f"Cluster {label}: {count} samples")

print("\nHierarchical cluster distribution:")
for label, count in zip(unique_hier, counts_hier):
    print(f"Cluster {label}: {count} samples")


KMeans cluster distribution:
Cluster 0: 28184 samples
Cluster 1: 13376 samples
Cluster 2: 28059 samples

Hierarchical cluster distribution:
Cluster 0: 2835 samples
Cluster 1: 2614 samples
Cluster 2: 1513 samples


**visualization**

In [23]:
# Total Sales by Shopping Mall
sales_by_mall=data.groupby('shopping_mall')['price'].sum().sort_values(ascending=False)

In [24]:
print(sales_by_mall.value_counts)

<bound method IndexOpsMixin.value_counts of shopping_mall
Mall of Istanbul     2621.170372
Kanyon               2594.409469
Metrocity            1939.355308
Metropol AVM         1312.707890
Istinye Park         1270.965726
Zorlu Center          664.110489
Cevahir AVM           649.707978
Viaport Outlet        646.037718
Emaar Square Mall     641.638581
Forum Istanbul        631.143217
Name: price, dtype: float64>


In [25]:
# Product Category Popularity
category_counts = data['category'].value_counts() #This counts how many times each category appears
print(category_counts)

category
Clothing           34487
Cosmetics          15097
Food & Beverage    14776
Toys               10087
Shoes              10034
Souvenir            4999
Technology          4996
Books               4981
Name: count, dtype: int64


In [26]:
monthly_sales = data.resample('M', on='invoice_date')['price'].sum()
#Group all purchases by month, and add up the total money earned each month

C:\Users\nadia\AppData\Local\Temp\ipykernel_14240\987097098.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = data.resample('M', on='invoice_date')['price'].sum()


In [27]:
print(monthly_sales)

invoice_date
2021-01-31    502.665652
2021-02-28    446.314658
2021-03-31    495.444443
2021-04-30    484.167866
2021-05-31    503.786608
2021-06-30    481.899996
2021-07-31    530.363059
2021-08-31    498.178572
2021-09-30    478.783966
2021-10-31    526.607977
2021-11-30    481.868377
2021-12-31    495.623246
2022-01-31    502.601668
2022-02-28    438.565136
2022-03-31    511.852369
2022-04-30    481.236098
2022-05-31    508.015913
2022-06-30    494.443507
2022-07-31    520.364973
2022-08-31    499.121782
2022-09-30    489.181852
2022-10-31    521.608126
2022-11-30    462.994147
2022-12-31    506.327071
2023-01-31    495.640592
2023-02-28    484.193866
2023-03-31    129.395228
Freq: ME, Name: price, dtype: float64


In [28]:
sns.set(style="darkgrid")
plt.figure(figsize=(18, 12))

#  Total Sales by Shopping Mall
plt.subplot(2, 2, 1)
sns.barplot(x=sales_by_mall.values, y=sales_by_mall.index, palette="rocket")
plt.title('Total Sales by Shopping Mall', fontsize=16)
plt.xlabel('Total Sales')
plt.ylabel('Shopping Mall')

# Product Category Popularity
plt.subplot(2,2,2)
myexplode = [0.2, 0, 0, 0,0,0,0,0]
plt.pie(x=category_counts, labels=category_counts.index, autopct='%1.1f%%', explode = myexplode, shadow = True,colors=sns.color_palette("flare"))
plt.title('Product Category Popularity', fontsize=16)

# Monthly Sales Trend (Line Chart)
plt.subplot(2, 1, 2)
monthly_sales.plot(marker='*', color='saddlebrown',mec = 'darkslategray')
plt.title('Monthly Sales Trend', fontsize=16)
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.grid(True)
plt.tight_layout()
plt.show()
#teal

C:\Users\nadia\AppData\Local\Temp\ipykernel_14240\2458728956.py:6: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=sales_by_mall.values, y=sales_by_mall.index, palette="rocket")
